In [55]:
from transformers import ElectraTokenizer, ElectraForQuestionAnswering
import torch

# 한국어 Electra 모델과 토크나이저 불러오기
model_name = "monologg/koelectra-base-v3-finetuned-korquad"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForQuestionAnswering.from_pretrained(model_name)


weather = ['날씨 정보를 알고 싶어요', '날씨를 물어보는 방법을 알려주세요', '날씨 서비스 이용하고 싶어요', '날씨기능 쓰는 법 좀 알려줘', '날씨 사용법 알려줘']

kakaotalk = ['카카오톡 메시지는 어떻게 보낼 수 있어요?', '카카오톡 메시지 기능은 어떻게 쓸 수 있나요?', '메시지 보내는 기능 좀 알려줘', '메시지는 어떻게 보낼 수 있어', '카카오톡 쓰는 법 좀']

parking = ['주차장 위치 서비스 쓰는 법 좀', '주차장 서비스 사용 방법 좀 알려주세요', '주차장 기능은 어떻게 쓰는 건가요?', '근처 주차장 찾고 싶은데 주차장 기능 좀 알려주세요', '주차장 서비스 이용하고 싶어요']

time = ['시간 서비스는 어떻게 이용할 수 있나요?', '시간 호출하는 방법 알려줘요', '시간 정보를 듣고 싶은데, 어떻게 하는거야', '현재 시간 기능 알려주는 기능 쓰고 싶어', '시간 기능에 대해서 알려주세요']

temperature = ['현재 온도에 대해서 알고 싶어요. 방법 좀 알려주세요', '온도를 알 수 있는 방법이 있나요?', '온도 기능을 쓰려면 어떻게 하죠?', '온도 기능 쓰는 방법 좀 알려주세요', '지금 온도 얼마인지 알고 싶어요']

humidity = ['습도 얼마인지 알고 싶은데, 어떻게 해요?', '습도 기능 이용법 좀 알려주세요', '오늘의 습도를 알고 싶어요', '습도 기능을 쓰고 싶은데, 방법 좀 알려주세요', '습도 기능 쓰고 싶어요']

error = ['영화 추천 기능 알려줘', '엔진 오일 얼마만큼 남았는지 알 수 있어?', '점심 추천 기능 있어', '알람 설정 기능 좀 알려줘', '오늘 활동량에 대해서 알고 싶어']

qst_lst = [weather, kakaotalk, parking, time, temperature, humidity, error]


print("질의 응답 시스템 성능 평가")
print()
for i in range(len(qst_lst)):
    for j in range(len(qst_lst[i])):
        # 예시 질문과 맥락
        question = qst_lst[i][j]
        context = """
        날씨 정보 서비스 기능을 이용하고 싶으면 weather로 한다.
        카카오톡 및 메시지 서비스 기능은 이용하고 싶으면 kakaotalk로 한다.
        주차장 위치 서비스 기능을 이용하고 싶으면 parking로 한다.
        시간 서비스 기능은 이용할 때는 time로 한다.
        현재 온도 서비스 기능를 이용하고 싶으면 temperature로 한다.
        현재 습도 서비스 기능을 이용하고 싶으면 humidity로 한다.
        """
        
        print("질문:", question)
        
        # 직접 모델 사용
        inputs = tokenizer(question, context, return_tensors="pt")
        outputs = model(**inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

        # argmax를 이용해 context에서 응답의 시작일 확률이 가장 높은 토큰의 위치를 반환
        answer_start = torch.argmax(answer_start_scores)
        # argmax를 이용해 context에서 응답의 끝일 확률이 가장 높은 토큰의 위치를 반환
        answer_end = torch.argmax(answer_end_scores) + 1

        # 토큰화 결과로부터 input_ids만 추출
        input_ids = inputs["input_ids"].tolist()[0]

        # input_ids에서 응답에 해당하는 id를 가져와 토큰으로 변환하고 다시 문자열로 변환
        answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
        answer = tokenizer.convert_tokens_to_string(answer_tokens)
        print("인식된 단어:", answer)

        # 응답 문장 출력
        greeting = "안녕하세요. 운전만해 가이드 '브이즈'입니다."
        if 'weather' in answer:
            answer = f"{greeting} 날씨 정보는 '브이즈' 호출 후 '날씨'라고 질문하면 현재 날씨 정보를 알려줍니다."
        elif 'kakaotalk' in answer:
            answer = f"{greeting} 카카오톡 메시지는 '브이즈' 호출 후 '카카오톡'을 말하고 특정 사용자 지정 후 메시지 내용을 말씀하시면 '운전만해' 앱이 메시지를 전송합니다."
        elif 'parking' in answer:
            answer = f"{greeting} 주차장 정보는 '브이즈' 호출 후 '주차장'이라고 말하면 현재 위치에서 가장 가까운 주차장의 위치를 알려줍니다."
        elif 'time' in answer:
            answer = f"{greeting} 시간 정보는 '브이즈' 호출 후 '시간'이라고 말하면 현재 시간에 대해서 알려줍니다."
        elif 'temperature' in answer:
            answer = f"{greeting} 현재 온도 정보는 '브이즈' 호출 후 '온도'라고 말하면 현재 온도에 대해서 알려줍니다."
        elif 'humidity' in answer:
            answer = f"{greeting} 현재 습도 정보는 '브이즈' 호출 후 '습도'라고 말하면 현재 습도에 대해서 알려줍니다."
        else:
            answer = f"죄송합니다. '운전만해' 앱에서 제공하지 않는 기능입니다."
        print("응답:", answer)
        print()
#         print("시작 위치 스코어:", float(torch.max(answer_start_scores)))
#         print("끝 위치 스코어:", float(torch.max(answer_end_scores)))
    print()

질의 응답 시스템 성능 평가

질문: 날씨 정보를 알고 싶어요
인식된 단어: weather
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 날씨 정보는 '브이즈' 호출 후 '날씨'라고 질문하면 현재 날씨 정보를 알려줍니다.

질문: 날씨를 물어보는 방법을 알려주세요
인식된 단어: weather
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 날씨 정보는 '브이즈' 호출 후 '날씨'라고 질문하면 현재 날씨 정보를 알려줍니다.

질문: 날씨 서비스 이용하고 싶어요
인식된 단어: weather
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 날씨 정보는 '브이즈' 호출 후 '날씨'라고 질문하면 현재 날씨 정보를 알려줍니다.

질문: 날씨기능 쓰는 법 좀 알려줘
인식된 단어: weather
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 날씨 정보는 '브이즈' 호출 후 '날씨'라고 질문하면 현재 날씨 정보를 알려줍니다.

질문: 날씨 사용법 알려줘
인식된 단어: weather
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 날씨 정보는 '브이즈' 호출 후 '날씨'라고 질문하면 현재 날씨 정보를 알려줍니다.


질문: 카카오톡 메시지는 어떻게 보낼 수 있어요?
인식된 단어: kakaotalk
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 카카오톡 메시지는 '브이즈' 호출 후 '카카오톡'을 말하고 특정 사용자 지정 후 메시지 내용을 말씀하시면 '운전만해' 앱이 메시지를 전송합니다.

질문: 카카오톡 메시지 기능은 어떻게 쓸 수 있나요?
인식된 단어: kakaotalk
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 카카오톡 메시지는 '브이즈' 호출 후 '카카오톡'을 말하고 특정 사용자 지정 후 메시지 내용을 말씀하시면 '운전만해' 앱이 메시지를 전송합니다.

질문: 메시지 보내는 기능 좀 알려줘
인식된 단어: kakaotalk
응답: 안녕하세요. 운전만해 가이드 '브이즈'입니다. 카카오톡 메시지는 '브이즈' 호출 후 '카카오톡'을 